In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re

In [2]:
data = pd.read_excel(r'C:\Users\cxy\OneDrive\桌面\回国\实习\7-29\小规模测试.xlsx')
df = data.reset_index().drop(columns='index')
df['医院名称'] = df['医院名称'].astype('string')
df

,医院名称
0,伊春市带岭区人民医院
1,大庆市中西医结合医院
2,大同区林源医院
3,杜尔伯特蒙古族自治县中医医院
4,杜尔伯特蒙古族自治县人民医院
5,杜蒙县妇幼保健医院
6,杜尔伯特蒙古族自治县妇幼保健计划生育服务中心
7,大庆市萨尔图区陈立军内科诊所
8,大庆油田四厂医院
9,大庆油田总医院集团南区医院


In [3]:
def resolve(url, headers_list):
    for i in range(5):  # 循环去请求网站
        headers = np.random.choice(headers_list)
        response = requests.get(url, headers=headers, timeout=20)
        if response.status_code == 200:
            break
    return response

In [38]:
headers_list = [
       {
        'user-agent': "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36"
    }, {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36 Edg/116.0.1938.76'
    }, {
        'user-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/115.0'
    }, {
        'user-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:58.0) Gecko/20100101 Firefox/58.0'
    }, {
        'user-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:88.0) Gecko/20100101 Firefox/88.0'
    }, {
        'user-agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.87 Safari/537.36 SE 2.X MetaSr 1.0'
    }, {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'
    }, {
        'user-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.54 Safari/537.36'
    }, {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.71 Safari/537.36 Core/1.94.202.400 QQBrowser/11.9.5355.400'
    }, {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36'
    }, {
        'user-agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Mobile Safari/537.36'
    }
]
headers = np.random.choice(headers_list)

In [20]:
df

,医院名称
0,伊春市带岭区人民医院
1,大庆市中西医结合医院
2,大同区林源医院
3,杜尔伯特蒙古族自治县中医医院
4,杜尔伯特蒙古族自治县人民医院
5,杜蒙县妇幼保健医院
6,杜尔伯特蒙古族自治县妇幼保健计划生育服务中心
7,大庆市萨尔图区陈立军内科诊所
8,大庆油田四厂医院
9,大庆油田总医院集团南区医院


In [21]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

def extract_e_id(html_content):
    """从HTML中提取e.id"""
    soup = BeautifulSoup(html_content, 'html.parser')
    script_tags = soup.find_all('script')
    
    for script in script_tags:
        if script.string:
            match = re.search(r"e\.id=(\d+);", script.string)
            if match:
                return match.group(1)
    return None

def get_hospital_name_and_url(query_name):
    """根据查询名称获取医院的名称和URL"""
    search_url = f'https://www.dayi.org.cn/search?keyword={query_name}'
    #headers = {'User-Agent': 'Mozilla/5.0'}
    
    # 第一步：搜索页面，提取e.id
    search_response = requests.get(search_url, headers=headers)
    if search_response.status_code == 200:
        e_id = extract_e_id(search_response.text)
        if e_id:
            # 第二步：使用e.id访问详细信息页面
            hospital_url = f'https://www.dayi.org.cn/hospital/{e_id}.html'
            detail_response = requests.get(hospital_url, headers=headers)
            if detail_response.status_code == 200:
                soup = BeautifulSoup(detail_response.text, 'html.parser')
                title_tag = soup.find('title')
                if title_tag:
                    title_text = title_tag.text.split('_')[0]
                    return query_name, title_text, hospital_url
    return query_name, '信息未找到', '信息未找到'

# 示例医院名称列表
hospital_names = df  # 这里替换为你的医院名称列表

# 处理每个医院名称并收集结果
results = [get_hospital_name_and_url(name) for name in hospital_names]

# 创建DataFrame
df_results = pd.DataFrame(results, columns=['查询名称', '医院名称', 'URL'])
df_results
# 输出结果到CSV文件
#df_results.to_csv('hospital_data.csv', index=False)
#print("数据已保存到CSV文件。")


,查询名称,医院名称,URL
0,医院名称,信息未找到,信息未找到


In [39]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_hospital_info(query_name):
    """根据查询名称获取医院详细信息"""
    search_url = f'https://www.dayi.org.cn/search?keyword={query_name}'
    #print(search_url)
    #headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}
    response = requests.get(search_url, headers=headers)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        print(soup)
        # 查找包含医院名称的链接
        first_result = soup.find('div', class_="node-name")
        print(first_result)
        if first_result:
            link_tag = first_result.find('a', href=True)
            if link_tag:
                hospital_name_extracted = link_tag.text.strip()  # 从链接中提取显示的医院名称
                e_id = link_tag['href'].split('/')[-1].split('.')[0]
                detail_url = f'https://www.dayi.org.cn/hospital/{e_id}.html'
                return query_name, hospital_name_extracted, detail_url
    return query_name, '未找到医院信息', '未找到链接'

# 示例：查询医院名称列表
hospital_names = ["大庆市中西医结合医院"]
results = [get_hospital_info(name) for name in hospital_names]

# 创建DataFrame并保存到CSV
df_results = pd.DataFrame(results, columns=['查询名称', '搜索到的医院名称', 'URL'])
df_results
#df_results.to_csv('hospital_data.csv', index=False)
#print("数据已保存到CSV文件。")


<!DOCTYPE html>

<html data-n-head="%7B%22lang%22:%7B%22ssr%22:%22en%22%7D%7D" data-n-head-ssr="" lang="en">
<head>
<meta content="05321585140bd80a92356af1bdf2bb02" name="baidu_union_verify"/>
<meta content="pmrgi33nmfuw4ir2ejsgc6ljfzxxezzomnxcelbcm52wszbchirgezldg42daobvgyytimruhazweyjvmzsgkyzymezgmmlcmnqwkyjcfqrhi2lnmvjwc5tfei5dcnrvgq4deobyg43dmmzvpu" name="domain_verify"/>
<meta content="J0WmTmHnuv" name="sogou_site_verification"/>
<meta content="jSmYA0CgLfTT1WPLa2V6" name="bytedance-verification-code"/>
<meta content="58E9FA9A81652BDE2981B2A6ACF883A6" name="msvalidate.01"/>
<meta content="b07d63286a03a0d49d8e77c06bc6f8b4_1666320174" name="shenma-site-verification"/>
<meta charset="utf-8" data-n-head="ssr"/><meta content="width=device-width, initial-scale=1" data-n-head="ssr" name="viewport"/><meta content="中国医药信息查询平台是中国权威医药数据库之一,建有海量的医学和药学内容。包括疾病,症状,药品,医生,医院,中药材,食品营养,针炙穴位,医美等30+数据库,拥有百科,视频,问答,病例,预约,咨询,便民购药等板块。为大众提供了专业,科学,可靠,实用的健康科普知识,让大众面对医疗健康信息不再迷茫。" data-hid="description" data-n-

,查询名称,搜索到的医院名称,URL
0,大庆市中西医结合医院,未找到医院信息,未找到链接


In [48]:
#成功用例1
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import pandas as pd
import time

def setup_driver():
    # 指定本地 ChromeDriver 路径
    path_to_chromedriver = r"D:\chromedriver-win64\chromedriver-win64\chromedriver.exe"  # 根据你的路径进行调整
    service = Service(executable_path=path_to_chromedriver)
    options = webdriver.ChromeOptions()
    options.headless = True  # 无界面模式
    driver = webdriver.Chrome(service=service, options=options)
    return driver

def fetch_hospital_info(query_name, driver):
    url = f'https://www.dayi.org.cn/search?keyword={query_name}'
    driver.get(url)
    
    time.sleep(5)  # 根据实际情况调整等待时间
    
    try:
        first_result = driver.find_element(By.CSS_SELECTOR, '.node-name a')
        hospital_name = first_result.text
        href = first_result.get_attribute('href')
        e_id = href.split('/')[-1].split('.')[0]
        detail_url = f'https://www.dayi.org.cn/hospital/{e_id}.html'
        return query_name, hospital_name, detail_url
    except Exception as e:
        print(f"Error fetching {query_name}: {e}")
        return query_name, '未找到医院信息', '未找到链接'

# 假设 df 是你的 DataFrame，并且有一个列 '医院名称'

driver = setup_driver()  # 初始化一次驱动器
results = [fetch_hospital_info(name, driver) for name in df['医院名称']]
driver.quit()  # 确保浏览器关闭

# 创建结果 DataFrame 并保存
results_df = pd.DataFrame(results, columns=['查询名称', '搜索到的医院名称', 'URL'])
results_df
#results_df.to_csv('hospital_search_results.csv', index=False)


,查询名称,搜索到的医院名称,URL
0,伊春市带岭区人民医院,巴中市恩阳区人民医院,https://www.dayi.org.cn/hospital/1156470.html
1,大庆市中西医结合医院,大庆市中西医结合医院,https://www.dayi.org.cn/hospital/1156894.html
2,大同区林源医院,联勤保障部队第九一Ｏ医院,https://www.dayi.org.cn/hospital/1137852.html
3,杜尔伯特蒙古族自治县中医医院,马关县中医医院,https://www.dayi.org.cn/hospital/2651.html
4,杜尔伯特蒙古族自治县人民医院,巴中市恩阳区人民医院,https://www.dayi.org.cn/hospital/1156470.html
5,杜蒙县妇幼保健医院,联勤保障部队第九一Ｏ医院,https://www.dayi.org.cn/hospital/1137852.html
6,杜尔伯特蒙古族自治县妇幼保健计划生育服务中心,阳新县妇幼保健院,https://www.dayi.org.cn/hospital/1156815.html
7,大庆市萨尔图区陈立军内科诊所,小诊所600元一颗的全瓷牙怎么样,https://www.dayi.org.cn/hospital/295766.html
8,大庆油田四厂医院,四川绵阳四〇四医院,https://www.dayi.org.cn/hospital/1151948.html
9,大庆油田总医院集团南区医院,中国贵航集团三〇二医院,https://www.dayi.org.cn/hospital/3713.html


In [47]:
df

,医院名称
0,伊春市带岭区人民医院
1,大庆市中西医结合医院
2,大同区林源医院
3,杜尔伯特蒙古族自治县中医医院
4,杜尔伯特蒙古族自治县人民医院
5,杜蒙县妇幼保健医院
6,杜尔伯特蒙古族自治县妇幼保健计划生育服务中心
7,大庆市萨尔图区陈立军内科诊所
8,大庆油田四厂医院
9,大庆油田总医院集团南区医院


In [33]:
search_url

'https://www.dayi.org.cn/search?keyword=林甸县慈爱医院'

In [8]:
def extract_e_id(content):
    """从页面内容中提取e.id值。"""
    match = re.search(r"e\.id=(\d+);", content)
    if match:
        return match.group(1)
    else:
        return None

def get_hospital_info(url, headers):
    """从构建的URL获取医院信息。"""
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # 从<title>提取医院名称
    title_tag = soup.find('title')
    if title_tag:
        title_text = title_tag.get_text()
        hospital_name_extracted = title_text.split('_')[0]
        print(title_text)
    else:
        hospital_name_extracted = 'NA'
    
    return hospital_name_extracted

In [26]:
def extract_e_id(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    script_tags = soup.find_all('script')
    
    for script in script_tags:
        if script.string:  # 检查 script 标签是否包含内容
            match = re.search(r"e\.id=(\d+);", script.string)
            if match:
                return match.group(1)  # 返回找到的第一个匹配的 e.id 值

    return None  # 如果没有找到匹配项，则返回 None
url='https://www.dayi.org.cn/search?keyword=大庆市中西医结合医院'
response = requests.get(url, headers=headers)

if response.status_code == 200:
    e_id = extract_e_id(response.text)
    if e_id:
        print("Extracted e.id:", e_id)
    else:
        print("e.id not found")
else:
    print("Failed to fetch page")

e.id not found


In [16]:
response = requests.get(url, headers=headers)
    
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
        
        # 抓取 <title> 标签中的医院名称
    title = soup.find('title').text if soup.find('title') else '标题未找到'
    title = title.split('_')[0]
title

'中国人民解放军总医院第一医学中心'

In [10]:
base_url = 'https://www.dayi.org.cn/search?keyword='
hospital_ids = []
hospital_names = []

for hospital_name in df["医院名称"]:
    search_url = f'{base_url}{hospital_name}'
    initial_response = requests.get(search_url, headers=headers)
    e_id = extract_e_id(initial_response.text)
    print(e_id)
    #print("Status Code:", initial_response.status_code)
    #print("Response Body:", initial_response.text[:500])  # 打印前500个字符

    if e_id:
        detailed_url = f"https://www.dayi.org.cn/hospital/{e_id}"
        extracted_name = get_hospital_info(detailed_url, headers)
        hospital_ids.append(e_id)
        hospital_names.append(extracted_name)
    else:
        hospital_ids.append('NA')
        hospital_names.append('NA')

# 创建新的DataFrame保存结果
new_df = pd.DataFrame({
    'Original Hospital Name': df["医院名称"],
    'Extracted Hospital Name': hospital_names,
    'eID': hospital_ids
})
new_df

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


,Original Hospital Name,Extracted Hospital Name,eID
0,伊春市带岭区人民医院,NA,NA
1,大庆市中西医结合医院,NA,NA
2,大同区林源医院,NA,NA
3,杜尔伯特蒙古族自治县中医医院,NA,NA
4,杜尔伯特蒙古族自治县人民医院,NA,NA
5,杜蒙县妇幼保健医院,NA,NA
6,杜尔伯特蒙古族自治县妇幼保健计划生育服务中心,NA,NA
7,大庆市萨尔图区陈立军内科诊所,NA,NA
8,大庆油田四厂医院,NA,NA
9,大庆油田总医院集团南区医院,NA,NA
